In [15]:
# load ASE atom from POSCAR file
from ase.io.vasp import read_vasp
import matplotlib.pyplot as plt
import os
import numpy as np
from glob import glob
import pickle

from ase import Atoms
from ase.neighborlist import NeighborList
from ase.build import make_supercell

from ovito.modifiers import CoordinationAnalysisModifier
from ovito.pipeline import StaticSource, Pipeline
from ovito.io.ase import ase_to_ovito

from mlptools.data.structures import generate_crystal
from mlptools.io.write import QuantumEspressoWriter

In [19]:
# randomly displace atoms while compressing or expanding the cell
path2root = '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/stishovite__mp-6947'
path2POSCAR = os.path.join(path2root, "original", "POSCAR")
path2save = os.path.join(path2root, "relax")
path2input = os.path.join(path2root, "input")

original_atoms = read_vasp(path2POSCAR)

supercell_atoms = make_supercell(original_atoms, [[2,0,0],[0,2,0],[0,0,2]])

print(f"Number of atoms in original: {len(original_atoms)}")
print(f"Number of atoms in supercell: {len(supercell_atoms)}")

Number of atoms in original: 6
Number of atoms in supercell: 48


In [17]:
with open(os.path.join(path2input, "relax.in")) as f:
    original_scf_input_lines = [s.strip() for s in f.readlines()]

for i, atoms in enumerate([supercell_atoms]):
    writer = QuantumEspressoWriter(atoms, path2input, scf_filename="relax.in")
    scf_input_lines = writer.output()

    # create dir
    path2result = f'{path2save}/relax_{i}'
    if not os.path.exists(path2result):
        os.makedirs(path2result)
    else:
        print(f'{path2result} already exists')

    # write scf.in
    with open(f'{path2result}/relax.in', 'w') as f:
        f.writelines('\n'.join(scf_input_lines))

# 計算完了済みのSCFを初期構造にして構造最適化計算を行う準備

In [4]:
# SiO2
# path2scf = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/result"
# path2relax = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/relax"
# path2input = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/input"
# path2outdir_root = "/home/y1u0d2/result/qe/SiO2/mp-546794/relax"

# num_samples = 350
# all_scf_dirs = glob(f"{path2scf}/scf_*")
# print(f"Number of scf dirs: {len(all_scf_dirs)}")

# sampled_scf_dirs = np.random.choice(all_scf_dirs, num_samples)

Number of scf dirs: 7697


In [17]:
# Si
from mlptools.db.db import get_session
from mlptools.db.model import Structure
from glob import glob
import pickle
import os

ATOMIC_SYMBOL = "Si"
PATH2ATOMS = "/Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms"

session = get_session(structure_symbol=ATOMIC_SYMBOL)
all_structure = session.query(Structure).filter(Structure.structure_id == "mp-149").all()

all_target_scf_dir = []
for structure in all_structure:
    path2target = os.path.join(PATH2ATOMS, structure.id, "atoms.pkl")
    if not os.path.exists(path2target):
        print(f"atoms.pkl not fount in {path2target}")
    atoms = pickle.load(open(path2target, "rb"))
    if atoms.n_atoms == 8:
        continue
    else:
        all_target_scf_dir.append(os.path.join(PATH2ATOMS, structure.id))
        print(f"[ADD] {path2target}")

[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/584802d0-bc22-4af3-9beb-9a1b3c81d34d/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/79c9721a-cd89-4664-9267-5e8fe47ee899/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/965ca21b-8364-4c19-aaf1-6ad72bf47874/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/ad02e271-201b-4d0f-abf0-cc892020d163/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/12928724-e1ad-41e6-bdf1-3f04d43ee2f3/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/607ad9c0-0bda-4556-8307-2fe2f9f598ba/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/c1c7df6a-2788-4103-bd2c-36565c4949fc/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/d769812d-8e57-4f85-9fbf-d4d601d963d7/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/8ab86bf1-3172-4427-851b-5538e0713c95/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/c82fb6e1-653e-4925-87c7-26f96af5b6c2/

In [18]:
path2relax = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result"
path2input = "/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/template"
path2outdir_root = "/home/y1u0d2/result/qe/Si/mp-149/relax/result"

num_samples = 350

sampled_scf_dirs = np.random.choice(all_target_scf_dir, num_samples)

In [19]:
# 全原子取得
for scf_dir in sampled_scf_dirs:
    atoms = pickle.load(open(f"{scf_dir}/atoms.pkl", "rb"))

    # create dir
    basename = os.path.basename(scf_dir)
    new_basename = f"relax_{basename}"
    path2result = f'{path2relax}/{new_basename}'
    if not os.path.exists(path2result):
        os.makedirs(path2result)
    else:
        print(f'{path2result} already exists')
    
    # get scf lines
    path2outdir = os.path.join(path2outdir_root, new_basename)
    writer = QuantumEspressoWriter(atoms.ase_atoms, path2input, scf_filename="relax.in", out_dir=path2outdir)
    scf_input_lines = writer.output()

    # write scf.in
    with open(f'{path2result}/relax.in', 'w') as f:
        f.writelines('\n'.join(scf_input_lines))

/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_10ecf750-b11f-46b2-9696-e36801117406 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_bd9f5137-80fd-4c7b-83db-e4adcbb5edd0 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_d426022d-c0a1-4bf2-800b-96c700b85bef already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_065a5437-9b44-4bd0-8739-729aae091f01 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_1bfa2d80-5183-4362-84cd-602aa3bfc67e already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_0dd50e04-f655-4b06-ba8a-8368bee6fbf0 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_b329ba7b-246e-4761-961d-8409fa6076de already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax_67ebde9b-dda7-442f-9ee9-5c5201c7ff6e already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/mp-149/relax/result/relax